In [1]:
# Execute if necessary
# %%capture
# !pip install numpy seaborn matplotlib pandas

In [2]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from typing import Dict, Tuple, Union, List

# Práctica 3: Modelo Lineal

__Instrucciones__: A continuación hay una lista de funciones que debe implementar o tareas que debe desarrollar. La descripción de cada una de ellas se encuentra en la definición de cada una de las funciones. Cada función está marcada por &#x1F625;,  &#x1F643; o &#x1F921;. Las marcas indican:

- &#x1F625;: Indican una entrega que debe ser hecha dentro de la misma sesión de la asignación. 
- &#x1F643;: Indican una entrega que puede ser hecha hasta la siguiente sesión.
- &#x1F921;: Debe mostrar un avance en la misma sesión, pero la entrega puede ser hecha en la siguiente.

Aquellas entregas parciales que no sean hechas el día de la asignación ya no serán válidas para las entregas totales, sin embargo, las entregas totales seguirán siendo válidas.

En esta sección se incluye un dataset real. El dataset importado incluye multiples características que describen las condiciones de los pasajeros en el accidente del titanic.

- __PassengerId__: Identificador de cada pasajero.
- __Survived__: 0 si no sobrevivió al accidente, 1 si lo hizo.
- __Pclass__: Clase en la que viajaba el pasajero, 1 - Primera clase, 2 - Segunda clase y 3 - Tercera clase.
- __Name__: Nombre del pasajero.
- __Sex__: Sexo del pasajero.
- __Age__: Edad del pasajero.
- __SibSp__: Número de hermanos más número de esposas con las que viajaba el pasajero.
- __Parch__: Número de padres más número de hijos con las que viajaba el pasajero.
- __Ticket__: Número de boleto.
- __Fare__: Tarifa del boleto del pasajero.
- __Cabin__: Número de cabina del pasajero.
- __Embarked__: Puerto de embarcación, C - Cherbourg, Q - Queenstown y S - Southampton.


In [3]:
df = pd.read_csv("titanic.csv")
df.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


## Asignación 1 &#x1F625;

Realice el preprocesamiento que considere adecuado para que las características __Pclass__, __Sex__, __SibSp__, __Parch__, __Fare__, __Cabin__, __Embarked__ y __Survived__ puedan ser utilizadas por un modelo lineal.

In [4]:
# Creamos un df con solo dichas columnas
data = df.drop(columns = ["PassengerId", "Name", "Age", "Ticket","Cabin"]) 

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   SibSp     891 non-null    int64  
 4   Parch     891 non-null    int64  
 5   Fare      891 non-null    float64
 6   Embarked  889 non-null    object 
dtypes: float64(1), int64(4), object(2)
memory usage: 48.9+ KB


In [6]:
# 2. Revisamos si existen nulos
data.isnull().sum()

Survived    0
Pclass      0
Sex         0
SibSp       0
Parch       0
Fare        0
Embarked    2
dtype: int64

In [7]:
# mostrar instancias con nulos
data[data["Embarked"].isna()]

,Survived,Pclass,Sex,SibSp,Parch,Fare,Embarked
61,1,1,female,0,0,80.0,NaN
829,1,1,female,0,0,80.0,NaN


In [8]:
# Eliminar nulos
df_limpio = data.dropna()
df_limpio.isnull().sum()

Survived    0
Pclass      0
Sex         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

In [9]:
# Convertir las características categóricas en variables dummy
sex_dummies = pd.get_dummies(df_limpio['Sex'], prefix='Sex')
embarked_dummies = pd.get_dummies(df_limpio['Embarked'], prefix='Embarked')
df_selected = pd.concat([df_limpio, sex_dummies, embarked_dummies], axis=1)
df_selected.drop(['Sex', 'Embarked'], axis=1, inplace=True)

# Normalizar las características numéricas
numeric_features = ['Pclass', 'SibSp', 'Parch', 'Fare']
for feature in numeric_features:
    feature_mean = df_selected[feature].mean()
    feature_std = df_selected[feature].std()
    df_selected[feature] = (df_selected[feature] - feature_mean) / feature_std

print(df_selected)

     Survived    Pclass     SibSp     Parch      Fare  Sex_female  Sex_male  \
0           0  0.824744  0.431108 -0.474059 -0.499958           0         1   
1           1 -1.571327  0.431108 -0.474059  0.788503           1         0   
2           1  0.824744 -0.474932 -0.474059 -0.486376           1         0   
3           1 -1.571327  0.431108 -0.474059  0.422623           1         0   
4           0  0.824744 -0.474932 -0.474059 -0.483861           0         1   
..        ...       ...       ...       ...       ...         ...       ...   
886         0 -0.373291 -0.474932 -0.474059 -0.384258           0         1   
887         1 -1.571327 -0.474932 -0.474059 -0.042189           1         0   
888         0  0.824744  0.431108  2.004991 -0.173986           1         0   
889         1 -1.571327 -0.474932 -0.474059 -0.042189           0         1   
890         0  0.824744 -0.474932 -0.474059 -0.489897           0         1   

     Embarked_C  Embarked_Q  Embarked_S  
0        

## Asignación 2 &#x1F625;

Utilizando las características __Pclass__, __Sex__, __SibSp__, __Parch__, __Fare__, __Cabin__ y __Embarked__, entrene un clasificador lineal para predecir __Survived__ utilizando el algoritmo _pocket_. Imprima el error obtenido.

In [10]:
# Paso 1: Importar bibliotecas
from sklearn.model_selection import train_test_split

In [11]:
data_ini = pd.read_csv("titanic.csv")
data_ini.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [17]:
data_dos = data_ini.drop(["PassengerId", "Name", "Ticket", "Cabin"], axis=1)
data_dos["Sex"] = pd.factorize(data_dos["Sex"])[0]
data_dos["Embarked"] = pd.factorize(data_dos["Embarked"])[0]
data_dos.fillna(data_dos.mean())

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,0,22.0,1,0,7.2500,0
1,1,1,1,38.0,1,0,71.2833,1
2,1,3,1,26.0,0,0,7.9250,0
3,1,1,1,35.0,1,0,53.1000,0
4,0,3,0,35.0,0,0,8.0500,0
...,...,...,...,...,...,...,...,...
886,0,2,0,27.0,0,0,13.0000,0
887,1,1,1,19.0,0,0,30.0000,0
888,0,3,1,NaN,1,2,23.4500,0
889,1,1,0,26.0,0,0,30.0000,1


In [18]:
# Paso 4: Separar datos en conjuntos de entrenamiento y prueba
X = data_dos.drop("Survived", axis=1)
y = data_dos["Survived"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [19]:
# Paso 5: Definir función de entrenamiento del clasificador
def train(X_train, y_train, X_test, y_test, max_iter=1000):
    # Agregar una columna de unos para el término de sesgo
    X_train = np.hstack((np.ones((X_train.shape[0], 1)), X_train))
    X_test = np.hstack((np.ones((X_test.shape[0], 1)), X_test))

    # Inicializar pesos aleatorios
    w = np.random.randn(X_train.shape[1])

    # Entrenar el clasificador con el algoritmo pocket
    best_w = w
    best_error = np.inf
    error = np.zeros(max_iter)
    for i in range(max_iter):
        y_pred = np.sign(np.dot(X_train, w))
        y_pred[y_pred == 0] = -1
        error[i] = np.mean(y_pred != y_train)

        if error[i] < best_error:
            best_w = w.copy()
            best_error = error[i]

        # Actualizar pesos con el algoritmo de perceptrón
        idx = np.where(y_pred != y_train)[0]
        if len(idx) == 0:
            break
        x = X_train[idx[0]]
        y_true = y_train[idx[0]]
        w = w + y_true * x

    # Calcular error en conjunto de prueba
    y_pred_test = np.sign(np.dot(X_test, best_w))
    y_pred_test[y_pred_test == 0] = -1
    test_error = np.mean(y_pred_test != y_test)

    return best_w, best_error, test_error, error

In [20]:
# Paso 6: Entrenar el clasificador y calcular error de entrenamiento y prueba
w, train_error, test_error, error = train(X_train, y_train, X_test, y_test)

# Imprimir resultados
print("Error de entrenamiento:", train_error)
print("Error de prueba:", test_error)

Error de entrenamiento: 1.0
Error de prueba: 0.994413407821229


## Asignación 3 &#x1F625;

Utilizando las características __Pclass__, __Sex__, __SibSp__, __Parch__, __Cabin__ y __Embarked__, entrene una regresión lineal para predecir __Fare__ utilizando el algoritmo de Ordinary Leasts Squares (OLS). Imprima el valor del error cuadrático medio (MSE).

In [16]:
# Selección de características y preprocesamiento de datos
selected_features = ['Pclass', 'Sex', 'SibSp', 'Parch', 'Cabin', 'Embarked', 'Fare']
df_selected = df[selected_features]

df_selected['CabinAssigned'] = df_selected['Cabin'].notnull().astype(int)

df_selected.drop(['Cabin'], axis=1, inplace=True)

sex_dummies = pd.get_dummies(df_selected['Sex'], prefix='Sex')
embarked_dummies = pd.get_dummies(df_selected['Embarked'], prefix='Embarked')
df_selected = pd.concat([df_selected, sex_dummies, embarked_dummies], axis=1)
df_selected.drop(['Sex', 'Embarked'], axis=1, inplace=True)

numeric_features = ['Pclass', 'SibSp', 'Parch', 'Fare']
for feature in numeric_features:
    feature_mean = df_selected[feature].mean()
    feature_std = df_selected[feature].std()
    df_selected[feature] = (df_selected[feature] - feature_mean) / feature_std

# División de los datos en conjunto de entrenamiento y prueba
train_size = int(len(df_selected) * 0.8)
train_set = df_selected[:train_size]
test_set = df_selected[train_size:]

X_train = train_set.drop('Fare', axis=1).values
y_train = train_set['Fare'].values
X_test = test_set.drop('Fare', axis=1).values
y_test = test_set['Fare'].values

# Ajuste de la regresión lineal con el algoritmo OLS
X_train = np.c_[np.ones(X_train.shape[0]), X_train]
coefficients = np.linalg.inv(X_train.T @ X_train) @ X_train.T @ y_train

# Evaluación del rendimiento en el conjunto de prueba
X_test = np.c_[np.ones(X_test.shape[0]), X_test]
y_pred = X_test @ coefficients
mse = np.mean((y_test - y_pred) ** 2)

print(f"Error cuadrático medio (MSE): {np.round(mse, 3)}")

Error cuadrático medio (MSE): 3.316


C:\Users\regin\AppData\Local\Temp\ipykernel_10488\2991516030.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected['CabinAssigned'] = df_selected['Cabin'].notnull().astype(int)
C:\Users\regin\AppData\Local\Temp\ipykernel_10488\2991516030.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected.drop(['Cabin'], axis=1, inplace=True)


## Asignación 4 &#x1F921;

Utilizando las características __Pclass__, __Sex__, __SibSp__, __Parch__, __Fare__, __Cabin__ y __Embarked__, entrene un clasificador lineal para predecir la probabilidad de supervivencia __Survived__ utilizando el algoritmo de gradiente descendente estocástico y la entropía cruzada como función de error. Imprima el arror en cada iteración del gradiente.